In [2]:
import numpy as np
import re
import heapq
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [3]:
# 1. Text Cleaner
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zñáéíóúü]", " ", newString)
    long_words=[]
    for i in newString.split():
        if len(i)>=3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [4]:

with open('training_data.txt', 'r', encoding='utf-8') as file:
    training_data = file.read()

data_new = text_cleaner(training_data)

In [5]:
# 2. Creating Sequences
def create_seq(text, length = 30):
    sequences = list()
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    return sequences

sequences = create_seq(data_new)

In [6]:
# 3. Character Mapping
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))
reverse_mapping = {i: c for c, i in mapping.items()}  # Reverse mapping for efficiency

def encode_seq(seq):
    sequences = list()
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
    return sequences

sequences = encode_seq(sequences)

In [7]:
# 4. Preparing the dataset
vocab = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab)
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# 5. Defining the Model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=Adam(learning_rate=0.01))

In [8]:
# 6. Training
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('model.h5', save_best_only=True, save_weights_only=False, monitor='val_loss'),
             ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)]

history = model.fit(X_tr, y_tr, epochs=50, batch_size=256, verbose=1, callbacks=callbacks, validation_data=(X_val, y_val))

Epoch 1/50
3960/3960 [==============================] - 33s 7ms/step - loss: 2.0249 - acc: 0.3872 - val_loss: 2.1130 - val_acc: 0.3496 - lr: 0.0100
Epoch 2/50
3960/3960 [==============================] - 27s 7ms/step - loss: 2.0996 - acc: 0.3480 - val_loss: 2.0491 - val_acc: 0.3702 - lr: 0.0100
Epoch 3/50
3960/3960 [==============================] - 27s 7ms/step - loss: 2.1223 - acc: 0.3395 - val_loss: 2.0597 - val_acc: 0.3584 - lr: 0.0100
Epoch 4/50
3960/3960 [==============================] - 27s 7ms/step - loss: 2.1174 - acc: 0.3406 - val_loss: 2.0585 - val_acc: 0.3668 - lr: 0.0100
Epoch 5/50
3960/3960 [==============================] - 28s 7ms/step - loss: 2.1075 - acc: 0.3495 - val_loss: 2.0695 - val_acc: 0.3585 - lr: 0.0100
Epoch 6/50
3960/3960 [==============================] - 27s 7ms/step - loss: 1.9897 - acc: 0.3834 - val_loss: 1.9174 - val_acc: 0.4119 - lr: 0.0020
Epoch 7/50
3960/3960 [==============================] - 27s 7ms/step - loss: 1.9693 - acc: 0.3924 - val_loss: 1.

In [9]:
# 7. Function for Text Generation using Beam Search
def generate_seq_beam_search(model, mapping, seq_length, seed_text, n_chars, beam_width=3):
    sequences = [{'seq': seed_text, 'score': 0.0}]
    for _ in range(n_chars):
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]['seq'], sequences[i]['score']
            encoded = [mapping[char] for char in seq]
            encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre').squeeze()
            pred = model.predict(np.array([encoded]), verbose=0)
            probas = np.exp(pred) / np.sum(np.exp(pred))
            top_k = heapq.nlargest(beam_width, zip(probas[0], list(range(len(probas[0])))))
            for j in range(len(top_k)):
                score_, idx = top_k[j]
                out_char = reverse_mapping[idx]
                candidate = {'seq': seq + out_char, 'score': score - np.log(score_)}
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup:tup['score'], reverse=True)
        sequences = ordered[:beam_width]
    return sequences

In [10]:
#Function Design for predicting missing text after a sentence
seed_text = "Juebes veinte tres de febrero de mil setecientos".lower()
num_chars_to_predict = 30
results = generate_seq_beam_search(model, mapping, 30, seed_text, num_chars_to_predict)
print(results[0]['seq'])

juebes veinte tres de febrero de mil setecientosadnoresce dronegcteneosacraces


In [11]:
#Predicting missing text in a gap 
# Define the text before and after the gap
text_before_gap = "Juebes veinte".lower()
text_after_gap = "de mil setecientos".lower()

# Estimate the gap length (this can be a rough estimate or based on context)
# Here, I'm using an arbitrary value; adjust as needed
gap_length_estimate = 15

# Generate content for the gap
results = generate_seq_beam_search(model, mapping, 30, text_before_gap, gap_length_estimate)
predicted_gap_content = results[0]['seq'][len(text_before_gap):]

# Merge the content
completed_sentence = text_before_gap + predicted_gap_content + text_after_gap
print(completed_sentence)

juebes veinterosellacle nterde mil setecientos


In [13]:
from sklearn.metrics import confusion_matrix, classification_report

# Extract unique classes from y_true and y_pred
unique_classes = np.union1d(y_true, y_pred)

# Create target names from these unique classes
target_names = [reverse_mapping[idx] for idx in unique_classes]

# Compute Precision, Recall, and Accuracy
report = classification_report(y_true, y_pred, labels=unique_classes, target_names=target_names)
print("\nClassification Report:")
print(report)



Classification Report:
              precision    recall  f1-score   support

                   0.54      0.72      0.62     16590
           a       0.34      0.37      0.35     10747
           b       0.53      0.24      0.33      1440
           c       0.30      0.16      0.21      4345
           d       0.25      0.20      0.23      3124
           e       0.40      0.37      0.39     10673
           f       0.36      0.23      0.28      1014
           g       0.47      0.33      0.39      1412
           h       0.54      0.26      0.35       861
           i       0.39      0.40      0.40      7961
           j       0.35      0.33      0.34       847
           k       0.25      0.06      0.10        34
           l       0.39      0.28      0.33      4415
           m       0.31      0.34      0.32      2953
           n       0.41      0.46      0.43      6568
           o       0.45      0.39      0.42      9577
           p       0.31      0.51      0.39      2507
   

C:\Users\Suzreal\miniconda3\envs\tf888\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Suzreal\miniconda3\envs\tf888\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Suzreal\miniconda3\envs\tf888\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
